# Thank you everyone!

This is a notbook of 25th solution. My solution is very simple.
1. I use the results of [public best submission](https://www.kaggle.com/ahmedewida/indoorlocation-ensembling).
2. Correct floor prediction based on the leakages of [shared wifi records](https://www.kaggle.com/tomooinubushi/retrieving-user-id-from-leaked-wifi-feature) and [device IDs](https://www.kaggle.com/c/indoor-location-navigation/discussion/234543)
3. Postprocess the waypoints based on [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) to search the path with minimal [cost](https://www.kaggle.com/saitodevel01/indoor-post-processing-by-cost-minimization)

I use public sub, because my NN model is never better than 7.7 in CV.

I use some codes and ideas from following notebooks. Thank you very much.

* https://www.kaggle.com/kenmatsu4/feature-store-for-indoor-location-navigation
* https://www.kaggle.com/jiweiliu/fix-the-timestamps-of-test-data-using-dask
* https://www.kaggle.com/ahmedewida/indoorlocation-ensembling
* https://www.kaggle.com/c/indoor-location-navigation/discussion/234543
* https://www.kaggle.com/saitodevel01/indoor-post-processing-by-cost-minimization
* https://www.kaggle.com/museas/with-magn-cost-minimization
* https://www.kaggle.com/higepon/visualize-submissions-with-post-processing

In [ ]:
import optuna
from scipy.sparse.csgraph import shortest_path
import matplotlib.pyplot
import numpy
import cv2
import argparse
from scipy.spatial import distance
from collections import OrderedDict
import warnings
import torch
from sklearn.utils.validation import _deprecate_positional_args
import os
import random
import time
import math
from collections import defaultdict
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.validation import check_is_fitted, column_or_1d
import seaborn as sns
from IPython.display import display  # Allows the use of display() for DataFrames
from pylab import imread
from IPython.display import Image
from matplotlib import animation as ani
from matplotlib_venn import venn2, venn2_circles
from matplotlib import rc
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import font_manager
import matplotlib
import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime
import dill
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from typing import Dict, List, Tuple, NamedTuple

import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.nn import CrossEntropyLoss, MSELoss
from torch.autograd import Variable
from scipy.ndimage import maximum_filter
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points
import shapely.ops as so
from descartes import PolygonPatch
import scipy.signal as signal
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks
ts_conv = np.vectorize(datetime.datetime.fromtimestamp)  # ut(10 digit) -> date

# pandas settings -----------------------------------------
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# Graph drawing -------------------------------------------

plt.rcParams["patch.force_edgecolor"] = True
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]


# ML -------------------------------------------

warnings.filterwarnings('ignore')

# import cudf
# import cupy as cp

# cfg = {
#     'experiment_name': 'jsmp',
#     'kaggle': False,
#     'seed': 42,
#     'now_str' : datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
# }
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d

In [ ]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj, f)


def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data


def pickle_dump_dill(obj, path):
    with open(path, mode='wb') as f:
        dill.dump(obj, f)


def pickle_load_dill(path):
    with open(path, mode='rb') as f:
        data = dill.load(f)
        return data

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.


def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(
        100 * (start_mem - end_mem) / start_mem))

    return df


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything()

data_dir = '../input/indoor-location-navigation/'
mydata_dir = '../input/indoordataset/'
out_dir = '../input/indoor-public/features/'

In [ ]:
class FeatureStore():

    # necessayr to re-check
    floor_convert = {'1F':  0, '2F': 1, '3F': 2, '4F': 3, '5F': 4,
                     '6F': 5, '7F': 6, '8F': 7, '9F': 8,
                     'B': -1, 'B1': -1, 'B2': -2, 'B3': -3,
                     'BF': -1, 'BM': -1,
                     'F1': 0, 'F2': 1, 'F3': 2, 'F4': 3, 'F5': 4,
                     'F6': 5, 'F7': 6, 'F8': 7, 'F9': 8, 'F10': 9,
                     'L1': 0, 'L2': 1, 'L3': 2, 'L4': 3, 'L5': 4,
                     'L6': 5, 'L7': 6, 'L8': 7, 'L9': 8, 'L10': 9,
                     'L11': 10,
                     'G': 0, 'LG1': 0, 'LG2': 1, 'LM': 0, 'M': 0,
                     'P1': 0, 'P2': 1, }

    df_types = ['accelerometer',
                'accelerometer_uncalibrated',
                'beacon',
                'gyroscope',
                'gyroscope_uncalibrated',
                'magnetic_field',
                'magnetic_field_uncalibrated',
                'rotation_vector',
                'waypoint',
                'wifi']

    # https://github.com/location-competition/indoor-location-competition-20
    df_type_cols = {'accelerometer': ["timestamp", "x", "y", "z", "accuracy"],
                    'accelerometer_uncalibrated': ["timestamp", "x", "y", "z",
                                                   "x2", "y2", "z2", "accuracy"],
                    'beacon': ["timestamp", "uuid", "major_id", "minor_id", "tx_power",
                               "rssi", "distance", "mac_addr", "timestamp2"],
                    'gyroscope': ["timestamp", "x", "y", "z", "accuracy"],
                    'gyroscope_uncalibrated': ["timestamp", "x", "y", "z",
                                               "x2", "y2", "z2", "accuracy"],
                    'magnetic_field': ["timestamp", "x", "y", "z", "accuracy"],
                    'magnetic_field_uncalibrated': ["timestamp", "x", "y", "z",
                                                    "x2", "y2", "z2", "accuracy"],
                    'rotation_vector': ["timestamp", "x", "y", "z", "accuracy"],
                    'waypoint': ["timestamp", "x", "y"],
                    'wifi': ["timestamp", "ssid", "bssid", "rssi", "frequency",
                             "last_seen_timestamp", ]}

    dtype_dict = {}
    dtype_dict["accelerometer"] = {"timestamp": int, "x": float, "y": float, "z": float,
                                   "accuracy": int}
    dtype_dict["accelerometer_uncalibrated"] = {"timestamp": int, "x": float, "y": float,
                                                "z": float, "x2": float, "y2": float,
                                                "z2": float, "accuracy": int}
    dtype_dict["beacon"] = {"timestamp": int, "uuid": str, "major_id": str,
                            "minor_id": str, "tx_power": int,  "rssi": int,
                            "distance": float, "mac_addr": str, "timestamp2": int}
    dtype_dict["gyroscope"] = {"timestamp": int, "x": float, "y": float, "z": float,
                               "accuracy": int}
    dtype_dict["gyroscope_uncalibrated"] = {"timestamp": int, "x": float, "y": float,
                                            "z": float, "x2": float, "y2": float,
                                            "z2": float, "accuracy": int}
    dtype_dict["magnetic_field"] = {"timestamp": int, "x": float, "y": float,
                                    "z": float, "accuracy": int}
    dtype_dict["magnetic_field_uncalibrated"] = {"timestamp": int, "x": float,
                                                 "y": float, "z": float, "x2": float,
                                                 "y2": float, "z2": float, "accuracy": int}
    dtype_dict["rotation_vector"] = {"timestamp": int, "x": float, "y": float,
                                     "z": float, "accuracy": int}
    dtype_dict["waypoint"] = {"timestamp": int,
                              "x": float, "y": float, "z": float}
    dtype_dict["wifi"] = {"timestamp": int, "ssid": str, "bssid": str,
                          "rssi": int, "frequency": int, "last_seen_timestamp": int}

    def __init__(self, site_id, floor, path_id,
                 input_path="../input/indoor-location-navigation/",
                 save_path="../mid"):
        self.site_id = site_id.strip()
        self.floor = floor.strip()
        self.n_floor = self.floor_convert[self.floor]
        self.path_id = path_id.strip()

        self.input_path = input_path
        assert Path(input_path).exists(
        ), f"input_path do not exist: {input_path}"

        self.save_path = save_path
        Path(save_path).mkdir(parents=True, exist_ok=True)

        self.site_info = SiteInfo(
            site_id=self.site_id, floor=self.floor, input_path=self.input_path)

    def _flatten(self, l):
        return list(itertools.chain.from_iterable(l))

    def multi_line_spliter(self, s):
        matches = re.finditer("TYPE_", s)
        matches_positions = [match.start() for match in matches]
        split_idx = [0] + [matches_positions[i] -
                           14 for i in range(1, len(matches_positions))] + [len(s)]
        return [s[split_idx[i]:split_idx[i+1]] for i in range(len(split_idx)-1)]

    def load_df(self, ):
        path = str(Path(self.input_path) /
                   f"train/{self.site_id}/{self.floor}/{self.path_id}.txt")
        with open(path) as f:
            data = f.readlines()

        modified_data = []
        for s in data:
            if s.count("TYPE_") > 1:
                lines = self.multi_line_spliter(s)
                modified_data.extend(lines)
            else:
                modified_data.append(s)
        del data
        self.meta_info_len = len([d for d in modified_data if d[0] == "#"])
        self.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":")
                                          for m in self._flatten([d.split("\t")
                                                                  for d in modified_data if d[0] == "#"]) if m != "#"])

        data_df = pd.DataFrame([d.replace("\n", "").split("\t")
                                for d in modified_data if d[0] != "#"])
        for dt in self.df_types:
            # select data type
            df_s = data_df[data_df[1] == f"TYPE_{dt.upper()}"]
            if len(df_s) == 0:
                setattr(self, dt, pd.DataFrame(columns=self.df_type_cols[dt]))
            else:
                # remove empty cols
                na_info = df_s.isna().sum(axis=0) == len(df_s)
                df_s = df_s[[i for i in na_info[na_info ==
                                                False].index if i != 1]].reset_index(drop=True)

                if len(df_s.columns) != len(self.df_type_cols[dt]):
                    df_s.columns = self.df_type_cols[dt][:len(df_s.columns)]
                else:
                    df_s.columns = self.df_type_cols[dt]

                # set dtype
                for c in df_s.columns:
                    df_s[c] = df_s[c].astype(self.dtype_dict[dt][c])

                # set DataFrame to attr
                setattr(self, dt, df_s)

    def get_site_info(self, keep_raw=False):
        self.site_info.get_site_info(keep_raw=keep_raw)

    def load_all_data(self, keep_raw=False):
        self.load_df()
        self.get_site_info(keep_raw=keep_raw)

    def __getitem__(self, item):
        if item in self.df_types:
            return getattr(self, item)
        else:
            return None

    def save(self, ):
        # to be implemented
        pass


class SiteInfo():
    def __init__(self, site_id, floor, input_path="../input/indoor-location-navigation/"):
        self.site_id = site_id
        self.floor = floor
        self.input_path = input_path
        assert Path(input_path).exists(
        ), f"input_path do not exist: {input_path}"

    def get_site_info(self, keep_raw=False):
        floor_info_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_info.json"
        with open(floor_info_path, "r") as f:
            self.floor_info = json.loads(f.read())
            self.site_height = self.floor_info["map_info"]["height"]
            self.site_width = self.floor_info["map_info"]["width"]
            if not keep_raw:
                del self.floor_info

        geojson_map_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/geojson_map.json"
        with open(geojson_map_path, "r") as f:
            self.geojson_map = json.loads(f.read())
            self.map_type = self.geojson_map["type"]
            self.features = self.geojson_map["features"]

            self.floor_coordinates = self.features[0]["geometry"]["coordinates"]
            self.store_coordinates = [self.features[i]["geometry"]["coordinates"]
                                      for i in range(1, len(self.features))]

            if not keep_raw:
                del self.geojson_map

    def show_site_image(self):
        path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_image.png"
        plt.imshow(imread(path), extent=[
                   0, self.site_width, 0, self.site_height])

    def draw_polygon(self, size=8, only_floor=False):

        fig = plt.figure()
        ax = plt.subplot(111)

        xmax, xmin, ymax, ymin = self._draw(
            self.floor_coordinates, ax, calc_minmax=True)
        if not only_floor:
            self._draw(self.store_coordinates, ax, fill=True)
        plt.legend([])

        xrange = xmax - xmin
        yrange = ymax - ymin
        ratio = yrange / xrange

        self.x_size = size
        self.y_size = size*ratio

        fig.set_figwidth(size)
        fig.set_figheight(size*ratio)
        # plt.show()
        return ax

    def _draw(self, coordinates, ax, fill=False, calc_minmax=False):
        xmax, ymax = -np.inf, -np.inf
        xmin, ymin = np.inf, np.inf
        for i in range(len(coordinates)):
            ndim = np.ndim(coordinates[i])
            if ndim == 2:
                corrd_df = pd.DataFrame(coordinates[i])
                if fill:
                    ax.fill(corrd_df[0], corrd_df[1], alpha=0.7)
                else:
                    corrd_df.plot.line(x=0, y=1, style="-", ax=ax)

                if calc_minmax:
                    xmax = max(xmax, corrd_df[0].max())
                    xmin = min(xmin, corrd_df[0].min())

                    ymax = max(ymax, corrd_df[1].max())
                    ymin = min(ymin, corrd_df[1].min())
            elif ndim == 3:
                for j in range(len(coordinates[i])):
                    corrd_df = pd.DataFrame(coordinates[i][j])
                    if fill:
                        ax.fill(corrd_df[0], corrd_df[1], alpha=0.6)
                    else:
                        corrd_df.plot.line(x=0, y=1, style="-", ax=ax)

                    if calc_minmax:
                        xmax = max(xmax, corrd_df[0].max())
                        xmin = min(xmin, corrd_df[0].min())

                        ymax = max(ymax, corrd_df[1].max())
                        ymin = min(ymin, corrd_df[1].min())
            else:
                assert False, f"ndim of coordinates should be 2 or 3: {ndim}"
        if calc_minmax:
            return xmax, xmin, ymax, ymin
        else:
            return None

In [ ]:
# train_meta_data
train_meta = glob(data_dir+"train/*/*/*")
train_meta_org = pd.DataFrame(train_meta)
train_meta = train_meta_org[0].str.split("/", expand=True)[[4, 5, 6]]
train_meta.columns = ["site_id", "floor", "path_id"]
train_meta["path_id"] = train_meta["path_id"].str.replace(".txt", "")
train_meta["path"] = train_meta_org[0]
train_meta.head()


# test_meta_data
sample_sub = pd.read_csv(data_dir+'sample_submission.csv')
test_meta = sample_sub["site_path_timestamp"].apply(
    lambda x: pd.Series(x.split("_")))
test_meta.columns = ["site_id", "path_id", "timestamp"]
test_meta = test_meta.drop('timestamp', axis=1)
test_meta = test_meta.drop_duplicates(
    subset=["site_id", "path_id"]).reset_index(drop=True)

In [ ]:
def read_txt(file):
    with open(file) as f:
        txt = f.readlines()

    modified_data = []
    for s in txt:
        if s.count("TYPE_") > 1:
            lines = multi_line_spliter(s)
            modified_data.extend(lines)
        else:
            modified_data.append(s)
    return modified_data


def _flatten(l):
    return list(itertools.chain.from_iterable(l))


def get_feature_test(site_id, path_id, input_path, sample_sub):
    file = f"{input_path}/test/{path_id}.txt"
    content = read_txt(file)
    data_df = pd.DataFrame([d.replace("\n", "").split("\t")
                            for d in content if d[0] != "#"])
    data_dict = OrderedDict()
    for dt in FeatureStore.df_types:
        # select data type
        df_s = data_df[data_df[1] == f"TYPE_{dt.upper()}"]
        if len(df_s) == 0:
            setattr(data_dict, dt, pd.DataFrame(
                columns=FeatureStore.df_type_cols[dt]))
        else:
            # remove empty cols
            na_info = df_s.isna().sum(axis=0) == len(df_s)
            df_s = df_s[[i for i in na_info[na_info ==
                                            False].index if i != 1]].reset_index(drop=True)

            if len(df_s.columns) != len(FeatureStore.df_type_cols[dt]):
                df_s.columns = FeatureStore.df_type_cols[dt][:len(
                    df_s.columns)]
            else:
                df_s.columns = FeatureStore.df_type_cols[dt]

            # set dtype
            for c in df_s.columns:
                df_s[c] = df_s[c].astype(FeatureStore.dtype_dict[dt][c])
            setattr(data_dict, dt, df_s)
    data_dict.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":")
                                           for m in _flatten([d.split("\t")
                                                              for d in content if d[0] == "#"]) if m != "#"])
    data_dict.meta_info_df[data_dict.meta_info_df[0] == 'startTime'][1] = int(
        data_dict.meta_info_df[0][int(np.where(data_dict.meta_info_df[0] == 'startTime')[0]+1)])
    data_dict.meta_info_df[data_dict.meta_info_df[0] == 'endTime'][1] = int(
        data_dict.meta_info_df[0][int(np.where(data_dict.meta_info_df[0] == 'endTime')[0]+1)])

    data_dict.waypoint['timestamp'] = sample_sub[sample_sub.path_id ==
                                                 path_id].timestamp.values.astype(int)
    data_dict.waypoint['x'] = 0
    data_dict.waypoint['y'] = 0
    data_dict.n_floor = 0
    data_dict.site_id = site_id
    return data_dict

create_feature_pickle=False
if create_feature_pickle:
    print('create_feature_pickle')
    for i in tqdm(range(len(train_meta))):
        t = train_meta.iloc[i]
        #print(f"site_id: {t.site_id}, floor: {t.floor}, path_id: {t.path_id}")
        feature = FeatureStore(site_id=t.site_id, floor=t.floor,
                               path_id=t.path_id, input_path=data_dir)
        feature.load_all_data()
        pickle_dump_dill(feature, out_dir+'feature/'+t.path_id+'.pickle')

    for i in tqdm(range(len(test_meta))):
        t = test_meta.iloc[i]
        #print(f"site_id: {t.site_id}, floor: {t.floor}, path_id: {t.path_id}")
        feature = get_feature_test(
            site_id=t.site_id, path_id=t.path_id, input_path=data_dir, sample_sub=sample_sub)
        print(feature.meta_info_df[0][1])
        pickle_dump_dill(feature, out_dir+'feature/'+t.path_id+'.pickle')


def generate_count_rank(df, column):
    count_rank = df.groupby(column)[column].count().rank(
        ascending=False, method='first')
    df[column+'_count_rank'] = df[column].map(count_rank)
    #df[column+'_count_rank'] = df[column+'_count_rank'].astype(np.int32)
    return df, count_rank


def add_count_rank(df, count_rank_dict, column, mode='convert'):
    if mode == 'convert':
        df[column] = df[column].map(count_rank_dict.get)
    elif mode == 'add':
        df[column+'_count_rank'] = df[column].map(count_rank_dict.get)
    #df[column+'_count_rank'] = df[column+'_count_rank'].astype(np.int32)
    return df

In [ ]:
def fast_left_merge(df1, df2, key='timestamp'):
    df2 = df2.set_index(key)
    df = pd.concat([df1.reset_index(drop=True), df2.reindex(
        df1[key].values).reset_index(drop=True)], axis=1)
    return df


def drop_duplicate_wifi(wifi_df, last_seen_timestamp_margin=10):
    old_bssid = ''
    old_rssi = -9999
    old_last_seen_timestamp = -9999
    if len(wifi_df) > 0:
        new_rows = []
        wifi_df = wifi_df.sort_values(['bssid', 'timestamp'])
        for cnt, row in enumerate(wifi_df[['timestamp', 'ssid', 'bssid', 'rssi', 'frequency', 'last_seen_timestamp']].values):
            bssid = str(row[2])
            rssi = int(row[3])
            last_seen_timestamp = int(row[4])
            if (bssid != old_bssid) or (rssi != old_rssi) or (abs(last_seen_timestamp-old_last_seen_timestamp) > last_seen_timestamp_margin):
                new_rows.append(row)
            old_bssid = bssid
            old_rssi = rssi
            old_last_seen_timestamp = last_seen_timestamp
        new_rows = np.vstack(new_rows)
        new_wifi_df = pd.DataFrame(new_rows, columns=[
                                   'timestamp', 'ssid', 'bssid', 'rssi', 'frequency', 'last_seen_timestamp'])
        new_wifi_df = new_wifi_df.sort_values('timestamp')
    else:
        new_wifi_df = wifi_df
    return new_wifi_df

# https://github.com/location-competition/indoor-location-competition-20/blob/master/compute_f.py


def split_ts_seq(ts_seq, sep_ts):
    """
    :param ts_seq:
    :param sep_ts:
    :return:
    """
    tss = ts_seq[:, 0].astype(float)
    unique_sep_ts = np.unique(sep_ts)
    ts_seqs = []
    start_index = 0
    for i in range(0, unique_sep_ts.shape[0]):
        end_index = np.searchsorted(tss, unique_sep_ts[i], side='right')
        if start_index == end_index:
            continue
        ts_seqs.append(ts_seq[start_index:end_index, :].copy())
        start_index = end_index

    # tail data
    if start_index < ts_seq.shape[0]:
        ts_seqs.append(ts_seq[start_index:, :].copy())

    return ts_seqs


def correct_trajectory(original_xys, end_xy):
    """
    :param original_xys: numpy ndarray, shape(N, 2)
    :param end_xy: numpy ndarray, shape(1, 2)
    :return:
    """
    corrected_xys = np.zeros((0, 2))

    A = original_xys[0, :]
    B = end_xy
    Bp = original_xys[-1, :]

    angle_BAX = np.arctan2(B[1] - A[1], B[0] - A[0])
    angle_BpAX = np.arctan2(Bp[1] - A[1], Bp[0] - A[0])
    angle_BpAB = angle_BpAX - angle_BAX
    AB = np.sqrt(np.sum((B - A) ** 2))
    ABp = np.sqrt(np.sum((Bp - A) ** 2))

    corrected_xys = np.append(corrected_xys, [A], 0)
    for i in np.arange(1, np.size(original_xys, 0)):
        angle_CpAX = np.arctan2(
            original_xys[i, 1] - A[1], original_xys[i, 0] - A[0])

        angle_CAX = angle_CpAX - angle_BpAB

        ACp = np.sqrt(np.sum((original_xys[i, :] - A) ** 2))

        AC = ACp * AB / ABp

        delta_C = np.array([AC * np.cos(angle_CAX), AC * np.sin(angle_CAX)])

        C = delta_C + A

        corrected_xys = np.append(corrected_xys, [C], 0)

    return corrected_xys


def correct_positions(rel_positions, reference_positions):
    """
    :param rel_positions:
    :param reference_positions:
    :return:
    """
    rel_positions_list = split_ts_seq(rel_positions, reference_positions[:, 0])
    if len(rel_positions_list) != reference_positions.shape[0] - 1:
        # print(f'Rel positions list size: {len(rel_positions_list)}, ref positions size: {reference_positions.shape[0]}')
        del rel_positions_list[-1]
    assert len(rel_positions_list) == reference_positions.shape[0] - 1

    corrected_positions = np.zeros((0, 3))
    for i, rel_ps in enumerate(rel_positions_list):
        start_position = reference_positions[i]
        end_position = reference_positions[i + 1]
        abs_ps = np.zeros(rel_ps.shape)
        abs_ps[:, 0] = rel_ps[:, 0]
        # abs_ps[:, 1:3] = rel_ps[:, 1:3] + start_position[1:3]
        abs_ps[0, 1:3] = rel_ps[0, 1:3] + start_position[1:3]
        for j in range(1, rel_ps.shape[0]):
            abs_ps[j, 1:3] = abs_ps[j-1, 1:3] + rel_ps[j, 1:3]
        abs_ps = np.insert(abs_ps, 0, start_position, axis=0)
        corrected_xys = correct_trajectory(abs_ps[:, 1:3], end_position[1:3])
        corrected_ps = np.column_stack((abs_ps[:, 0], corrected_xys))
        if i == 0:
            corrected_positions = np.append(
                corrected_positions, corrected_ps, axis=0)
        else:
            corrected_positions = np.append(
                corrected_positions, corrected_ps[1:], axis=0)

    corrected_positions = np.array(corrected_positions)

    return corrected_positions


def init_parameters_filter(sample_freq, warmup_data, cut_off_freq=2):
    order = 4
    filter_b, filter_a = signal.butter(
        order, cut_off_freq / (sample_freq / 2), 'low', False)
    zf = signal.lfilter_zi(filter_b, filter_a)
    _, zf = signal.lfilter(filter_b, filter_a, warmup_data, zi=zf)
    _, filter_zf = signal.lfilter(filter_b, filter_a, warmup_data, zi=zf)

    return filter_b, filter_a, filter_zf


def get_rotation_matrix_from_vector(rotation_vector):
    q1 = rotation_vector[0]
    q2 = rotation_vector[1]
    q3 = rotation_vector[2]

    if rotation_vector.size >= 4:
        q0 = rotation_vector[3]
    else:
        q0 = 1 - q1*q1 - q2*q2 - q3*q3
        if q0 > 0:
            q0 = np.sqrt(q0)
        else:
            q0 = 0

    sq_q1 = 2 * q1 * q1
    sq_q2 = 2 * q2 * q2
    sq_q3 = 2 * q3 * q3
    q1_q2 = 2 * q1 * q2
    q3_q0 = 2 * q3 * q0
    q1_q3 = 2 * q1 * q3
    q2_q0 = 2 * q2 * q0
    q2_q3 = 2 * q2 * q3
    q1_q0 = 2 * q1 * q0

    R = np.zeros((9,))
    if R.size == 9:
        R[0] = 1 - sq_q2 - sq_q3
        R[1] = q1_q2 - q3_q0
        R[2] = q1_q3 + q2_q0

        R[3] = q1_q2 + q3_q0
        R[4] = 1 - sq_q1 - sq_q3
        R[5] = q2_q3 - q1_q0

        R[6] = q1_q3 - q2_q0
        R[7] = q2_q3 + q1_q0
        R[8] = 1 - sq_q1 - sq_q2

        R = np.reshape(R, (3, 3))
    elif R.size == 16:
        R[0] = 1 - sq_q2 - sq_q3
        R[1] = q1_q2 - q3_q0
        R[2] = q1_q3 + q2_q0
        R[3] = 0.0

        R[4] = q1_q2 + q3_q0
        R[5] = 1 - sq_q1 - sq_q3
        R[6] = q2_q3 - q1_q0
        R[7] = 0.0

        R[8] = q1_q3 - q2_q0
        R[9] = q2_q3 + q1_q0
        R[10] = 1 - sq_q1 - sq_q2
        R[11] = 0.0

        R[12] = R[13] = R[14] = 0.0
        R[15] = 1.0

        R = np.reshape(R, (4, 4))

    return R


def get_orientation(R):
    flat_R = R.flatten()
    values = np.zeros((3,))
    if np.size(flat_R) == 9:
        values[0] = np.arctan2(flat_R[1], flat_R[4])
        values[1] = np.arcsin(-flat_R[7])
        values[2] = np.arctan2(-flat_R[6], flat_R[8])
    else:
        values[0] = np.arctan2(flat_R[1], flat_R[5])
        values[1] = np.arcsin(-flat_R[9])
        values[2] = np.arctan2(-flat_R[8], flat_R[10])

    return values


def compute_steps(acce_datas):
    step_timestamps = np.array([])
    step_indexs = np.array([], dtype=int)
    step_acce_max_mins = np.zeros((0, 4))
    sample_freq = 50
    window_size = 22
    low_acce_mag = 0.6
    step_criterion = 1
    interval_threshold = 250

    acce_max = np.zeros((2,))
    acce_min = np.zeros((2,))
    acce_binarys = np.zeros((window_size,), dtype=int)
    acce_mag_pre = 0
    state_flag = 0

    warmup_data = np.ones((window_size,)) * 9.81
    filter_b, filter_a, filter_zf = init_parameters_filter(
        sample_freq, warmup_data)
    acce_mag_window = np.zeros((window_size, 1))

    # detect steps according to acceleration magnitudes
    for i in np.arange(0, np.size(acce_datas, 0)):
        acce_data = acce_datas[i, :]
        acce_mag = np.sqrt(np.sum(acce_data[1:] ** 2))

        acce_mag_filt, filter_zf = signal.lfilter(
            filter_b, filter_a, [acce_mag], zi=filter_zf)
        acce_mag_filt = acce_mag_filt[0]

        acce_mag_window = np.append(acce_mag_window, [acce_mag_filt])
        acce_mag_window = np.delete(acce_mag_window, 0)
        mean_gravity = np.mean(acce_mag_window)
        acce_std = np.std(acce_mag_window)
        mag_threshold = np.max([low_acce_mag, 0.4 * acce_std])

        # detect valid peak or valley of acceleration magnitudes
        acce_mag_filt_detrend = acce_mag_filt - mean_gravity
        if acce_mag_filt_detrend > np.max([acce_mag_pre, mag_threshold]):
            # peak
            acce_binarys = np.append(acce_binarys, [1])
            acce_binarys = np.delete(acce_binarys, 0)
        elif acce_mag_filt_detrend < np.min([acce_mag_pre, -mag_threshold]):
            # valley
            acce_binarys = np.append(acce_binarys, [-1])
            acce_binarys = np.delete(acce_binarys, 0)
        else:
            # between peak and valley
            acce_binarys = np.append(acce_binarys, [0])
            acce_binarys = np.delete(acce_binarys, 0)

        if (acce_binarys[-1] == 0) and (acce_binarys[-2] == 1):
            if state_flag == 0:
                acce_max[:] = acce_data[0], acce_mag_filt
                state_flag = 1
            elif (state_flag == 1) and ((acce_data[0] - acce_max[0]) <= interval_threshold) and (
                    acce_mag_filt > acce_max[1]):
                acce_max[:] = acce_data[0], acce_mag_filt
            elif (state_flag == 2) and ((acce_data[0] - acce_max[0]) > interval_threshold):
                acce_max[:] = acce_data[0], acce_mag_filt
                state_flag = 1

        # choose reasonable step criterion and check if there is a valid step
        # save step acceleration data: step_acce_max_mins = [timestamp, max, min, variance]
        step_flag = False
        if step_criterion == 2:
            if (acce_binarys[-1] == -1) and ((acce_binarys[-2] == 1) or (acce_binarys[-2] == 0)):
                step_flag = True
        elif step_criterion == 3:
            if (acce_binarys[-1] == -1) and (acce_binarys[-2] == 0) and (np.sum(acce_binarys[:-2]) > 1):
                step_flag = True
        else:
            if (acce_binarys[-1] == 0) and acce_binarys[-2] == -1:
                if (state_flag == 1) and ((acce_data[0] - acce_min[0]) > interval_threshold):
                    acce_min[:] = acce_data[0], acce_mag_filt
                    state_flag = 2
                    step_flag = True
                elif (state_flag == 2) and ((acce_data[0] - acce_min[0]) <= interval_threshold) and (
                        acce_mag_filt < acce_min[1]):
                    acce_min[:] = acce_data[0], acce_mag_filt
        if step_flag:
            step_timestamps = np.append(step_timestamps, acce_data[0])
            step_indexs = np.append(step_indexs, [i])
            step_acce_max_mins = np.append(step_acce_max_mins,
                                           [[acce_data[0], acce_max[1], acce_min[1], acce_std ** 2]], axis=0)
        acce_mag_pre = acce_mag_filt_detrend

    return step_timestamps, step_indexs, step_acce_max_mins


def compute_stride_length(step_acce_max_mins):
    K = 0.4
    K_max = 0.8
    K_min = 0.4
    para_a0 = 0.21468084
    para_a1 = 0.09154517
    para_a2 = 0.02301998

    stride_lengths = np.zeros((step_acce_max_mins.shape[0], 2))
    k_real = np.zeros((step_acce_max_mins.shape[0], 2))
    step_timeperiod = np.zeros((step_acce_max_mins.shape[0] - 1, ))
    stride_lengths[:, 0] = step_acce_max_mins[:, 0]
    window_size = 2
    step_timeperiod_temp = np.zeros((0, ))

    # calculate every step period - step_timeperiod unit: second
    for i in range(0, step_timeperiod.shape[0]):
        step_timeperiod_data = (
            step_acce_max_mins[i + 1, 0] - step_acce_max_mins[i, 0]) / 1000
        step_timeperiod_temp = np.append(
            step_timeperiod_temp, [step_timeperiod_data])
        if step_timeperiod_temp.shape[0] > window_size:
            step_timeperiod_temp = np.delete(step_timeperiod_temp, [0])
        step_timeperiod[i] = np.sum(
            step_timeperiod_temp) / step_timeperiod_temp.shape[0]

    # calculate parameters by step period and acceleration magnitude variance
    k_real[:, 0] = step_acce_max_mins[:, 0]
    k_real[0, 1] = K
    for i in range(0, step_timeperiod.shape[0]):
        k_real[i + 1, 1] = np.max([(para_a0 + para_a1 / step_timeperiod[i] +
                                    para_a2 * step_acce_max_mins[i, 3]), K_min])
        k_real[i + 1, 1] = np.min([k_real[i + 1, 1], K_max]) * (K / K_min)

    # calculate every stride length by parameters and max and min data of acceleration magnitude
    stride_lengths[:, 1] = np.max([(step_acce_max_mins[:, 1] - step_acce_max_mins[:, 2]),
                                   np.ones((step_acce_max_mins.shape[0], ))], axis=0)**(1 / 4) * k_real[:, 1]

    return stride_lengths


def compute_headings(ahrs_datas):
    headings = np.zeros((np.size(ahrs_datas, 0), 2))
    for i in np.arange(0, np.size(ahrs_datas, 0)):
        ahrs_data = ahrs_datas[i, :]
        rot_mat = get_rotation_matrix_from_vector(ahrs_data[1:])
        azimuth, pitch, roll = get_orientation(rot_mat)
        around_z = (-azimuth) % (2 * np.pi)
        headings[i, :] = ahrs_data[0], around_z
    return headings


def compute_step_heading(step_timestamps, headings):
    step_headings = np.zeros((len(step_timestamps), 2))
    step_timestamps_index = 0
    for i in range(0, len(headings)):
        if step_timestamps_index < len(step_timestamps):
            if headings[i, 0] == step_timestamps[step_timestamps_index]:
                step_headings[step_timestamps_index, :] = headings[i, :]
                step_timestamps_index += 1
        else:
            break
    assert step_timestamps_index == len(step_timestamps)

    return step_headings


def compute_rel_positions(stride_lengths, step_headings):
    rel_positions = np.zeros((stride_lengths.shape[0], 3))
    for i in range(0, stride_lengths.shape[0]):
        rel_positions[i, 0] = stride_lengths[i, 0]
        rel_positions[i, 1] = -stride_lengths[i, 1] * \
            np.sin(step_headings[i, 1])
        rel_positions[i, 2] = stride_lengths[i, 1] * \
            np.cos(step_headings[i, 1])

    return rel_positions


def compute_step_positions(acce_datas, ahrs_datas, posi_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_steps(
        acce_datas)
    headings = compute_headings(ahrs_datas)
    stride_lengths = compute_stride_length(step_acce_max_mins)
    step_headings = compute_step_heading(step_timestamps, headings)
    rel_positions = compute_rel_positions(stride_lengths, step_headings)
    step_positions = correct_positions(rel_positions, posi_datas)
    return step_positions


order = 3
fs = 50.0  # sample rate, Hz
# fs = 100
# cutoff = 3.667  # desired cutoff frequency of the filter, Hz
cutoff = 3

step_distance = 0.8
w_height = 1.7
m_trans = -5


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


def peak_accel_threshold(data, timestamps, threshold):
    d_acc = []
    last_state = 'below'
    crest_troughs = 0
    crossings = []

    for i, datum in enumerate(data):

        current_state = last_state
        if datum < threshold:
            current_state = 'below'
        elif datum > threshold:
            current_state = 'above'

        if current_state is not last_state:
            if current_state is 'above':
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)
            else:
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)

            crest_troughs += 1
        last_state = current_state
    return np.array(crossings)


def steps_compute_rel_positions(acce, magn):

    mix_acce = np.sqrt(acce[:, 1:2]**2 + acce[:, 2:3]**2 + acce[:, 3:4]**2)
    mix_acce = np.concatenate([acce[:, 0:1], mix_acce], 1)
    mix_df = pd.DataFrame(mix_acce)
    mix_df.columns = ["timestamp", "acce"]

    filtered = butter_lowpass_filter(mix_df["acce"], cutoff, fs, order)

    threshold = filtered.mean() * 1.1
    crossings = peak_accel_threshold(filtered, mix_df["timestamp"], threshold)

    step_sum = len(crossings)/2
    distance = w_height * 0.4 * step_sum

    mag_df = pd.DataFrame(magn)
    mag_df.columns = ["timestamp", "x", "y", "z"]

    acce_df = pd.DataFrame(acce)
    acce_df.columns = ["timestamp", "ax", "ay", "az"]

    mag_df = pd.merge(mag_df, acce_df, on="timestamp")
    mag_df.dropna()

    time_di_list = []

    for i in mag_df.iterrows():

        gx, gy, gz = i[1][1], i[1][2], i[1][3]
        ax, ay, az = i[1][4], i[1][5], i[1][6]

        roll = math.atan2(ay, az)
        pitch = math.atan2(-1*ax, (ay * math.sin(roll) + az * math.cos(roll)))

        q = m_trans - math.degrees(math.atan2(
            (gz*math.sin(roll)-gy*math.cos(roll)), (gx*math.cos(pitch) + gy *
                                                    math.sin(roll)*math.sin(pitch) + gz*math.sin(pitch)*math.cos(roll))
        )) - 90
        if q <= 0:
            q += 360
        time_di_list.append((i[1][0], q))

    d_list = [x[1] for x in time_di_list]

    steps = []
    step_time = []
    di_dict = dict(time_di_list)

    for n, i in enumerate(crossings[:, :1]):
        if n % 2 == 1:
            continue
        direct_now = di_dict[i[0]]
        dx = math.sin(math.radians(direct_now))
        dy = math.cos(math.radians(direct_now))
#         print(int(n/2+1),"歩目/x:",dx,"/y:",dy,"/角度：",direct_now)
        steps.append((i[0], dx, dy))
        step_time.append(i[0])

        step_dtime = np.diff(step_time)/1000
        step_dtime = step_dtime.tolist()
        step_dtime.insert(0, 5)

        rel_position = []

        wp_idx = 0
#         print("WP:",round(sample_file.waypoint[0,1],3),round(sample_file.waypoint[0,2],3),sample_file.waypoint[0,0])
#         print("------------------")
        for p, i in enumerate(steps):
            step_distance = 0
            if step_dtime[p] >= 1:
                step_distance = w_height*0.25
            elif step_dtime[p] >= 0.75:
                step_distance = w_height*0.3
            elif step_dtime[p] >= 0.5:
                step_distance = w_height*0.4
            elif step_dtime[p] >= 0.35:
                step_distance = w_height*0.45
            elif step_dtime[p] >= 0.2:
                step_distance = w_height*0.5
            else:
                step_distance = w_height*0.4

#             step_x += i[1]*step_distance
#             step_y += i[2]*step_distance

            rel_position.append([i[0], i[1]*step_distance, i[2]*step_distance])
#     print(rel_position)

    return np.array(rel_position)

In [ ]:
# Helper Functions
import pandas as pd
import numpy as np

import json
import matplotlib.pylab as plt
import random
def sub_process(sub, train_waypoints):
    sub = split_col(sub[['site_path_timestamp','floor','x','y']]).copy()
    sub = sub.merge(train_waypoints[['site','floorNo','floor']].drop_duplicates(), how='left')
    return sub
def split_col(df):
    df = pd.concat([
        df['site_path_timestamp'].str.split('_', expand=True) \
        .rename(columns={0:'site',
                         1:'path',
                         2:'timestamp'}),
        df
    ], axis=1).copy()
    return df

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}


def plot_preds(
    ax,
    context_text,
    site,
    floorNo,
    sub=None,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=True,
    show_preds=True,
    fix_labels=True,
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")


    ax.imshow(floor_img)

    if show_train:
        true_locs = true_locs.query('site == @site and floorNo == @map_floor').copy()
        true_locs["x_"] = true_locs["x"] * floor_img.shape[0] / height_meter
        true_locs["y_"] = (
            true_locs["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        true_locs.query("site == @site and floorNo == @map_floor").groupby("path").plot(
            x="x_",
            y="y_",
            style="+",
            ax=ax,
            label="train waypoint location",
            color="grey",
            alpha=0.5,
        )

    if show_preds:
        sub = sub.query('site == @site and floorNo == @floorNo').copy()
        sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
        sub["y_"] = (
            sub["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        for path, path_data in sub.query(
            "site == @site and floorNo == @floorNo"
        ).groupby("path"):
            path_data.plot(
                x="x_",
                y="y_",
                style=".-",
                ax=ax,
                title=f"{context_text} - {site} - floor - {floorNo}",
                alpha=1,
                label=path,
            )
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
    return

from pathlib import Path
def generate_target_sites(sub_df):
    return sorted(sub_df['site'].unique())

def generate_site_floors_dict(sub_df):
    sites = generate_target_sites(sub_df)
    site_floors_dict = {}
    for site in sites:
        site_path = Path('/kaggle/input/indoor-location-navigation/train') / site
        site_floors_dict[site] = [path.name for path in site_path.glob('*')]
    return site_floors_dict


def plot_sites(sites, sub1_df, sub2_df, sub3_df):
    num_floors = 0
    for site in sites:
        num_floors += len(site_floors_dict[site])

    fig, ax = plt.subplots(num_floors, 3, figsize=(42, 12 * num_floors))

    idx = 0
    for site in sites:
        floors = site_floors_dict[site]

        for floor in floors:
            plot_preds(ax[idx][0], "raw submission", site, floor, sub1_df, train_waypoints, show_preds=True)
            plot_preds(ax[idx][1], "Postprocess without augmentation", site, floor, sub2_df, train_waypoints, show_preds=True)
            plot_preds(ax[idx][2], "Postprocess with augmentation", site, floor, sub3_df, train_waypoints, show_preds=True)
            idx += 1
    plt.show()



# Generate augmented waypoints
* Augmented waypoints have similar X and Y values of train ones.
* Augmented waypoints are in hallways.
* Augmented waypoints are sufficiently distant from train ones and each other.


In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points
import shapely.ops as so
from descartes import PolygonPatch

    
def normalize_array(array, floor_max, floor_min, site_max):
    array=(array-floor_min)/(floor_max-floor_min)* site_max
    return array

create_polygon_dict=False
if create_polygon_dict:

    polygon_dict=defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: Polygon([]))))
    for i in tqdm(range(len(train_meta))):
        t = train_meta.iloc[i]
        n_floor = FeatureStore.floor_convert[t.floor]
        if polygon_dict[t.site_id][n_floor]['floor_polygons'] == Polygon([]):
            feature = pickle_load_dill(out_dir+'features/'+path_id+'.pickle')
            floor_arrays=np.zeros([0,2])
            for j in range(len(feature.site_info.floor_coordinates)):
                floor_array=np.array(feature.site_info.floor_coordinates[j]).squeeze()
                floor_arrays=np.vstack([floor_arrays,floor_array])
            floor_min=floor_arrays.min(axis=0)
            floor_max=floor_arrays.max(axis=0)
            site_max=np.array([feature.site_info.site_width,feature.site_info.site_height])

            floor_polygons=[]
            for j in range(len(feature.site_info.floor_coordinates)):
                floor_array=np.array(feature.site_info.floor_coordinates[j]).squeeze()
                floor_array = normalize_array(floor_array, floor_max, floor_min, site_max)
                floor_polygon = Polygon(floor_array)
                floor_polygons.append(floor_polygon)
            floor_polygons= so.unary_union(floor_polygons)

            store_polygons=[]
            for j in range(len(feature.site_info.store_coordinates)):
                for k in range(len(feature.site_info.store_coordinates[j])):
                    store_array=np.array(feature.site_info.store_coordinates[j][k]).squeeze()
                    store_array = normalize_array(store_array, floor_max, floor_min, site_max)
                    store_polygon  = Polygon(store_array)
                    store_polygons.append(store_polygon)
            store_polygons= so.unary_union(store_polygons)

            polygon_dict[t.site_id][feature.n_floor]['floor_polygons']=floor_polygons
            polygon_dict[t.site_id][feature.n_floor]['store_polygons']=store_polygons
else:
    polygon_dict=pickle_load_dill('../input/indoordataset/polygon_dict.pickle')

In [ ]:
def subsample_arr(arr, threshold=1):
    new_arr=[]
    old_x=-9999
    temp_list=[]
    for x in np.sort(arr):
        if len(temp_list)>0:
            temp_mean=np.concatenate([temp_list]).mean()
            if x-temp_mean > threshold:
                new_arr.append(temp_mean)
                temp_list=[]
        temp_list.append(x)
    if len(temp_list)>0:
        temp_mean=np.concatenate([temp_list]).mean()
        new_arr.append(temp_mean)
        
    new_arr=np.concatenate([new_arr])
    return new_arr

def rot_arr(arr, deg):
    rad=np.deg2rad(deg)
    rot_mat = np.array([[np.cos(rad), -np.sin(rad)],
                  [np.sin(rad),  np.cos(rad)]])
    return  np.dot(arr,rot_mat)

In [ ]:
def generate_aug_wps(wps,site_id,n_floor,polygon_dict,floor_margin=0,store_margin=0,dist_threshold=4,dist_to_wps_threshold=6,grid_threshold=1,num_arr=300, rot_grid=True, aug_generator='grid'):
    if aug_generator=='random':
        aug_wps=np.vstack([np.random.random_sample((num_arr))*(wps.max(axis=0)[0]-wps.min(axis=0)[0])+wps.min(axis=0)[0],np.random.random_sample((num_arr))*(wps.max(axis=0)[1]-wps.min(axis=0)[1])+wps.min(axis=0)[1]]).transpose()
    if aug_generator=='grid':
        if rot_grid:
            min_num_grid=np.inf
            for angle in range(90):
                rot_wps=rot_arr(wps, angle)
                wps_x=subsample_arr(rot_wps[:,0], threshold=grid_threshold)
                wps_y=subsample_arr(rot_wps[:,1], threshold=grid_threshold)
                len_wps_x=len(wps_x)
                len_wps_y=len(wps_y)
                num_grid=len_wps_x*len_wps_y
                if num_grid<min_num_grid:
                    min_num_grid=num_grid
                    min_angle=angle
            rot_wps=rot_arr(wps, min_angle)
            wps_x=subsample_arr(rot_wps[:,0], threshold=grid_threshold)
            wps_y=subsample_arr(rot_wps[:,1], threshold=grid_threshold)
            len_wps_x=len(wps_x)
            len_wps_y=len(wps_y)
            aug_wps=np.vstack([np.tile(wps_x, (len_wps_y)),np.repeat(wps_y, len_wps_x, axis=0)]).transpose()
            aug_wps=rot_arr(aug_wps, -min_angle)
        else:
            wps_x=subsample_arr(wps[:,0], threshold=grid_threshold)
            wps_y=subsample_arr(wps[:,1], threshold=grid_threshold)
            len_wps_x=len(wps_x)
            len_wps_y=len(wps_y)
            aug_wps=np.vstack([np.tile(wps_x, (len_wps_y)),np.repeat(wps_y, len_wps_x, axis=0)]).transpose()
    if aug_generator=='triangle':
        aug_wps=wps_sub.copy()
        tri = Delaunay(aug_wps)
        aug_wps=(np.vstack(aug_wps[tri.simplices])+np.vstack(np.roll(aug_wps[tri.simplices],1, axis=1)))/2
        idx = np.random.randint(len(aug_wps), size=100)
        aug_wps=aug_wps[idx,:]
        getunique
        sorted_idx = np.lexsort(aug_wps.T)
        sorted_data =  aug_wps[sorted_idx,:]
        # Get unique row mask
        row_mask = np.append([True],np.any(np.diff(sorted_data,axis=0),1))
        # Get unique rows
        aug_wps = sorted_data[row_mask]   
    floor_polygons=polygon_dict[site_id][n_floor]['floor_polygons']
    store_polygons=polygon_dict[site_id][n_floor]['store_polygons']
    floor_polygons_eroded = floor_polygons.buffer(-floor_margin)
    store_polygons_dilated = store_polygons.buffer(store_margin)
    safe_area_polygons=floor_polygons_eroded.difference(store_polygons_dilated)
    mask_arr=np.array([True]*len(aug_wps))
    for cnt, row in enumerate(aug_wps):
        x = float(row[0])
        y = float(row[1])
        point = Point([x,y])
        if not safe_area_polygons.contains(point):
            mask_arr[cnt]=False
    #print(aug_wps.shape)
    aug_wps=aug_wps[mask_arr]
    #print(aug_wps.shape)
    
    #remove aug wps based on distance to wps
    if len(aug_wps)>0:        
        dist_aug = distance.cdist(aug_wps, wps, metric='euclidean')
        dist_aug=dist_aug.min(axis=1)
        #aug_wps=aug_wps[(dist_aug<7)&(dist_aug>2)]
        aug_wps=aug_wps[dist_aug>dist_to_wps_threshold]
        
    #remove aug wps based on distance among aug wps
    if len(aug_wps)>0:
        dist_aug = distance.cdist(aug_wps, aug_wps, metric='euclidean')
        dist_aug = np.tril(dist_aug, k=-1)
        dist_aug = np.where(dist_aug == 0, np.inf, dist_aug) 
        for i in range(len(dist_aug)):
            if dist_aug[i,:].min() < dist_threshold:
                dist_aug[:,i]=np.inf    
        dist_aug=dist_aug.min(axis=1)
        #aug_wps=aug_wps[(dist_aug<7)&(dist_aug>2)]
        aug_wps=aug_wps[dist_aug>dist_threshold]
        
    #print(aug_wps.shape)
    return aug_wps

In [ ]:
wps_dict = pickle_load_dill(mydata_dir+'wps_dict.pickle')
train_meta_sub = pickle_load_dill(mydata_dir+'train_meta_sub.pickle')
test_meta = pickle_load_dill(mydata_dir+'test_meta.pickle')
#hlwps_df = pd.read_csv('../input/indoor-navigation-hand-labeled-waypoints/waypoint_by_hand.csv')


In [ ]:
create_aug_wps_dict=True
if create_aug_wps_dict:
    aug_wps_dict=defaultdict(lambda: defaultdict(lambda: np.empty([0, 2])))
    for site_id in tqdm(wps_dict.keys()):
        for n_floor in wps_dict[site_id].keys():
            wps=wps_dict[site_id][n_floor]
            aug_wps_dict[site_id][n_floor] = generate_aug_wps(wps,site_id,n_floor,polygon_dict,floor_margin=0,store_margin=0,dist_threshold=4,dist_to_wps_threshold=6,grid_threshold=1,rot_grid=False,num_arr=300, aug_generator='grid')
    pickle_dump_dill(aug_wps_dict,'aug_wps_dict.pickle')
else:
    aug_wps_dict = pickle_load_dill(mydata_dir+'aug_wps_dict.pickle')

# Correct floor prediction based on the leakages of [shared wifi records](https://www.kaggle.com/tomooinubushi/retrieving-user-id-from-leaked-wifi-feature) and [device IDs](https://www.kaggle.com/c/indoor-location-navigation/discussion/234543)

In [ ]:
create_df_user = False
if create_df_user:
    print('create_df_user')
    train_meta['start_time'] = 0
    train_meta['end_time'] = 0
    train_meta['start_wp_time'] = 0
    train_meta['start_wp_x'] = 0
    train_meta['start_wp_y'] = 0
    train_meta['end_wp_time'] = 0
    train_meta['end_wp_x'] = 0
    train_meta['end_wp_y'] = 0
    train_meta['n_floor'] = 0
    wifi_dict = defaultdict(lambda: pd.DataFrame())
    mag_unc_dict = defaultdict(lambda: pd.DataFrame())
    gyr_unc_dict = defaultdict(lambda: pd.DataFrame())
    acc_unc_dict = defaultdict(lambda: pd.DataFrame())
    for i in tqdm(range(len(train_meta))):
        t = train_meta.iloc[i]
        feature = pickle_load_dill(out_dir+'feature/'+t.path_id+'.pickle')
        if feature.meta_info_df[feature.meta_info_df[0] == 'startTime'][1].values == None:
            start_time = int(np.nanmin([feature.accelerometer.timestamp.min(
            ), feature.wifi.timestamp.min(), feature.beacon.timestamp.min()]))
        else:
            start_time = int(
                feature.meta_info_df[feature.meta_info_df[0] == 'startTime'][1])
        if (len(feature.meta_info_df[feature.meta_info_df[0] == 'endTime']) == 0) or (feature.meta_info_df[feature.meta_info_df[0] == 'endTime'][1].values == None):
            end_time = int(np.nanmax([feature.accelerometer.timestamp.max(
            ), feature.wifi.timestamp.max(), feature.beacon.timestamp.max()]))
        else:
            end_time = int(
                feature.meta_info_df[feature.meta_info_df[0] == 'endTime'][1])
        waypoint_df = feature.waypoint.copy()
        train_meta.loc[i, 'start_time'] = start_time
        train_meta.loc[i,
                       'start_wp_time'] = waypoint_df.iloc[0]['timestamp']
        train_meta.loc[i, 'start_wp_x'] = waypoint_df.iloc[0]['x']
        train_meta.loc[i, 'start_wp_y'] = waypoint_df.iloc[0]['y']
        train_meta.loc[i, 'end_time'] = end_time
        train_meta.loc[i,
                       'end_wp_time'] = waypoint_df.iloc[-1]['timestamp']
        train_meta.loc[i, 'end_wp_x'] = waypoint_df.iloc[-1]['x']
        train_meta.loc[i, 'end_wp_y'] = waypoint_df.iloc[-1]['y']
        train_meta.loc[i, 'n_floor'] = feature.n_floor

        wifi_dict[t.path_id] = feature.wifi[[
            'bssid', 'last_seen_timestamp']].drop_duplicates()
        if 'x2' in feature.accelerometer_uncalibrated.columns:
            acc_unc_dict[t.path_id] = feature.accelerometer_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()
        if 'x2' in feature.gyroscope_uncalibrated.columns:
            gyr_unc_dict[t.path_id] = feature.gyroscope_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()
        if 'x2' in feature.magnetic_field_uncalibrated.columns:
            mag_unc_dict[t.path_id] = feature.magnetic_field_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()
    train_meta = train_meta.sort_values(
        ['site_id', 'start_time']).reset_index(drop=True)

    test_meta['start_time'] = 0
    test_meta['end_time'] = 0
    test_meta['start_wp_time'] = 0
    test_meta['start_wp_x'] = 0
    test_meta['start_wp_y'] = 0
    test_meta['end_wp_time'] = 0
    test_meta['end_wp_x'] = 0
    test_meta['end_wp_y'] = 0
    test_meta['n_floor'] = 0
    for i in tqdm(range(len(test_meta))):
        t = test_meta.iloc[i]
        #print(f"site_id: {t.site_id}, floor: {t.floor}, path_id: {t.path_id}")
        feature = pickle_load_dill(out_dir+'feature/'+t.path_id+'.pickle')
        if feature.meta_info_df[feature.meta_info_df[0] == 'startTime'][1].values == None:
            start_time = int(np.nanmin([feature.accelerometer.timestamp.min(
            ), feature.wifi.timestamp.min(), feature.beacon.timestamp.min()]))
        else:
            start_time = int(
                feature.meta_info_df[feature.meta_info_df[0] == 'startTime'][1])
        if (len(feature.meta_info_df[feature.meta_info_df[0] == 'endTime']) == 0) or (feature.meta_info_df[feature.meta_info_df[0] == 'endTime'][1].values == None):
            end_time = int(np.nanmax([feature.accelerometer.timestamp.max(
            ), feature.wifi.timestamp.max(), feature.beacon.timestamp.max()]))
        else:
            end_time = int(
                feature.meta_info_df[feature.meta_info_df[0] == 'endTime'][1])
        if len(feature.beacon) > 0:
            gap = feature.beacon.loc[0, 'timestamp2'] - \
                feature.beacon.loc[0, 'timestamp']
        else:
            gap = (feature.wifi.last_seen_timestamp.values -
                   feature.wifi.timestamp.values).max()+324.5229450792481  # from mean gap
        waypoint_df = feature.waypoint.copy()
        test_meta.loc[i, 'start_time'] = start_time+gap
        test_meta.loc[i,
                      'start_wp_time'] = waypoint_df.iloc[0]['timestamp']
        test_meta.loc[i, 'start_wp_x'] = waypoint_df.iloc[0]['x']
        test_meta.loc[i, 'start_wp_y'] = waypoint_df.iloc[0]['y']
        test_meta.loc[i, 'end_time'] = end_time+gap
        test_meta.loc[i,
                      'end_wp_time'] = waypoint_df.iloc[-1]['timestamp']
        test_meta.loc[i, 'end_wp_x'] = waypoint_df.iloc[-1]['x']
        test_meta.loc[i, 'end_wp_y'] = waypoint_df.iloc[-1]['y']
        test_meta.loc[i, 'n_floor'] = feature.n_floor

        wifi_dict[t.path_id] = feature.wifi[[
            'bssid', 'last_seen_timestamp']].drop_duplicates()
        if 'x2' in feature.accelerometer_uncalibrated.columns:
            acc_unc_dict[t.path_id] = feature.accelerometer_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()
        if 'x2' in feature.gyroscope_uncalibrated.columns:
            gyr_unc_dict[t.path_id] = feature.gyroscope_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()
        if 'x2' in feature.magnetic_field_uncalibrated.columns:
            mag_unc_dict[t.path_id] = feature.magnetic_field_uncalibrated[[
                'x2', 'y2', 'z2']].drop_duplicates()

    df_user = pd.merge(train_meta, test_meta, how='outer')
    df_user = df_user.sort_values(
        ['site_id', 'start_time']).reset_index(drop=True)
    print('create_user_id')
    threshold = 0.00001
    df_user['user_id_wifi'] = 0
    df_user['counter_wifi'] = 0
    df_user['user_id_acc'] = 0
    df_user['counter_acc'] = 0
    df_user['min_dist_acc'] = 0
    df_user['user_id_gyr'] = 0
    df_user['counter_gyr'] = 0
    df_user['min_dist_gyr'] = 0
    df_user['user_id_mag'] = 0
    df_user['counter_mag'] = 0
    df_user['min_dist_mag'] = 0
    n_wifi = 0
    n_acc = 0
    n_gyr = 0
    n_mag = 0
    for i in tqdm(range(len(df_user))):
        t = df_user.iloc[i]
        current_wifi = wifi_dict[t.path_id]
        min_last_seen_timestamp = current_wifi.last_seen_timestamp.min()
        df_site = df_user[df_user.site_id == t.site_id]
        df_site = df_site[df_site.end_time < t.start_time]
        df_site = df_site[min_last_seen_timestamp < df_site.end_time]
        counter_wifi = 0
        counter_acc = 0
        counter_gyr = 0
        counter_mag = 0
        min_dist_acc = np.inf
        min_dist_gyr = np.inf
        min_dist_mag = np.inf
        if len(df_site) > 0:
            acc = acc_unc_dict[t.path_id]
            mag = mag_unc_dict[t.path_id]
            gyr = gyr_unc_dict[t.path_id]
            for j in range(len(df_site)):
                t = df_site.iloc[j]
                old_wifi = wifi_dict[t.path_id]
                common_wifi = pd.merge(
                    current_wifi, old_wifi, how='inner', on=['bssid'])
                common_wifi['diff_time'] = abs(
                    common_wifi.last_seen_timestamp_x-common_wifi.last_seen_timestamp_y)
                if (common_wifi.diff_time < 5).sum() > 0:
                    # If there is a leak
                    df_user.loc[i, 'user_id_wifi'] = t.user_id_wifi
                    counter_wifi += 1
            if len(acc) > 0:
                if acc.values.sum() > 0:
                    for j in range(len(df_site)):
                        t = df_site.iloc[j]
                        if len(acc_unc_dict[t.path_id]) > 0:
                            dist_M = distance.cdist(
                                acc.values, acc_unc_dict[t.path_id].values, metric='euclidean')
                            min_dist_acc = np.min([min_dist_acc, dist_M.min()])
                            if dist_M.min() < threshold:
                                df_user.loc[i, 'user_id_acc'] = t.user_id_acc
                                counter_acc += 1
            if len(mag) > 0:
                for j in range(len(df_site)):
                    t = df_site.iloc[j]
                    if len(mag_unc_dict[t.path_id]) > 0:
                        dist_M = distance.cdist(
                            mag.values, mag_unc_dict[t.path_id].values, metric='euclidean')
                        min_dist_mag = np.min([min_dist_mag, dist_M.min()])
                        if dist_M.min() < threshold:
                            df_user.loc[i, 'user_id_mag'] = t.user_id_mag
                            counter_mag += 1
            if len(gyr) > 0:
                for j in range(len(df_site)):
                    t = df_site.iloc[j]
                    if len(gyr_unc_dict[t.path_id]) > 0:
                        dist_M = distance.cdist(
                            gyr.values, gyr_unc_dict[t.path_id].values, metric='euclidean')
                        min_dist_gyr = np.min([min_dist_gyr, dist_M.min()])
                        if dist_M.min() < threshold:
                            df_user.loc[i, 'user_id_gyr'] = t.user_id_gyr
                            counter_gyr += 1
        if counter_wifi == 0:
            df_user.loc[i, 'user_id_wifi'] = n_wifi
            n_wifi += 1
        df_user.loc[i, 'counter_wifi'] = counter_wifi

        if counter_acc == 0:
            df_user.loc[i, 'user_id_acc'] = n_acc
            n_acc += 1
        df_user.loc[i, 'counter_acc'] = counter_acc
        df_user.loc[i, 'min_dist_acc'] = min_dist_acc

        if counter_mag == 0:
            df_user.loc[i, 'user_id_mag'] = n_mag
            n_mag += 1
        df_user.loc[i, 'counter_mag'] = counter_mag
        df_user.loc[i, 'min_dist_mag'] = min_dist_mag

        if counter_gyr == 0:
            df_user.loc[i, 'user_id_gyr'] = n_gyr
            n_gyr += 1
        df_user.loc[i, 'counter_gyr'] = counter_gyr
        df_user.loc[i, 'min_dist_gyr'] = min_dist_gyr

    df_user['user_id'] = df_user.user_id_wifi.copy()
    for user_id in df_user.user_id_acc.unique():
        df_user.loc[df_user.user_id_acc == user_id,
                    'user_id'] = df_user.loc[df_user.user_id_acc == user_id, 'user_id'].min()
    for user_id in df_user.user_id_mag.unique():
        df_user.loc[df_user.user_id_mag == user_id,
                    'user_id'] = df_user.loc[df_user.user_id_mag == user_id, 'user_id'].min()
    for user_id in df_user.user_id_gyr.unique():
        df_user.loc[df_user.user_id_gyr == user_id,
                    'user_id'] = df_user.loc[df_user.user_id_gyr == user_id, 'user_id'].min()
    for user_id in df_user.user_id_wifi.unique():
        df_user.loc[df_user.user_id_wifi == user_id,
                    'user_id'] = df_user.loc[df_user.user_id_wifi == user_id, 'user_id'].min()
    le = LabelEncoder()
    labels = df_user.user_id
    le.fit(labels)
    df_user.user_id = le.transform(labels)

    count_user_id_dict = defaultdict(int)
    for cnt, row in enumerate(df_user[['start_wp_x', 'user_id']].values):
        start_wp_x = int(row[0])
        user_id = int(row[1])
        if start_wp_x > 0:
            count_user_id_dict[user_id] += 1
        else:
            count_user_id_dict[user_id] = count_user_id_dict[user_id]
    count_rank_user_id_dict = {key: rank for rank, key in enumerate(
        sorted(count_user_id_dict, key=count_user_id_dict.get, reverse=True), 1)}
    if not cfg.debug:
        df_user.to_csv(out_dir + 'df_user_id.csv')
        pickle_dump_dill(count_user_id_dict, out_dir +
                         'count_user_id_dict.pickle')
        pickle_dump_dill(count_rank_user_id_dict, out_dir +
                         'count_rank_user_id_dict.pickle')
else:
    df_user = pd.read_csv(mydata_dir + 'df_user_id.csv')

In [ ]:
def correct_pred_leak(submission_df,df_user,nigel_pred_df,correct_start_end=True,correct_floor=True, if_all_count_one='nigel',time_threshold_site=10000,time_threshold_user=10000):
    out_df=submission_df.copy()
    df_user_train=df_user[df_user.start_wp_x>0]
    counter_floor = 0
    counter_start = 0
    counter_end = 0
    tmp = out_df['site_path_timestamp'].apply(
        lambda s: pd.Series(s.split('_')))
    out_df['site'] = tmp[0]
    out_df['path'] = tmp[1]
    out_df['timestamp'] = tmp[2].astype(float)
    
    tmp = nigel_pred_df['site_path_timestamp'].apply(
        lambda s: pd.Series(s.split('_')))
    nigel_pred_df['site'] = tmp[0]
    nigel_pred_df['path'] = tmp[1]
    nigel_pred_df['timestamp'] = tmp[2].astype(float)
    
    for path_id in tqdm(out_df.path.unique()):
        site_id=out_df[out_df.path==path_id].site.values[0]
        user_id=int(df_user[df_user.path_id==path_id].user_id)
        pred_floor=out_df[out_df.path==path_id].floor.values[0]
        start_time=int(df_user[df_user.path_id==path_id].start_time)
        end_time=int(df_user[df_user.path_id==path_id].end_time)
        
        if correct_floor:
            nigel_pred=int(nigel_pred_df[nigel_pred_df.path==path_id].floor.iloc[0])
            corrected_floors=[nigel_pred]
            gap_time_end=0
            gap_time_start=0

            user_df=df_user_train[df_user_train.user_id==user_id]
            site_df=df_user_train[df_user_train.site_id==site_id]
            
            if len(user_df)>0:        
                uniqs, counts = np.unique(user_df.n_floor.values, return_counts=True)
                user_df_floor_mode=int(uniqs[counts == np.amax(counts)][0])
                corrected_floors.append(user_df_floor_mode)
                
            user_df_end = user_df[user_df.end_time < start_time]
            if len(user_df_end) > 0:
                nearest_endpoint = user_df_end.loc[user_df_end.end_time.idxmax()]
                corrected_floors.append(nearest_endpoint.n_floor)
            else:   
                site_df_end = site_df[(site_df.end_time < start_time)&(start_time-site_df.end_time<time_threshold_site)]
                if len(site_df_end) > 0:
                    nearest_endpoint = site_df_end.loc[site_df_end.end_time.idxmax()]
                    gap_time_end=start_time-nearest_endpoint.end_time
                    corrected_floors.append(nearest_endpoint.n_floor)
                    
            user_df_start = user_df[end_time < user_df.start_time]
            if len(user_df_start) > 0:
                nearest_startpoint = user_df_start.loc[user_df_start.start_time.idxmin()]
                corrected_floors.append(nearest_startpoint.n_floor)
            else:
                site_df_start = site_df[(end_time < site_df.start_time)&(site_df.start_time-end_time<time_threshold_site)]        
                if len(site_df_start) > 0:
                    nearest_startpoint = site_df_start.loc[site_df_start.start_time.idxmin()]
                    gap_time_start=nearest_startpoint.start_time-end_time
                    corrected_floors.append(nearest_startpoint.n_floor)

            corrected_floors=np.array(corrected_floors)
            uniqs, counts = np.unique(corrected_floors, return_counts=True)
            corrected_floor=int(uniqs[counts == np.amax(counts)][0])

            if (counts==1).all():
                if if_all_count_one == 'nigel':
                    corrected_floor = nigel_pred
                if if_all_count_one == 'leak':
                    if len(user_df)>0:
                        corrected_floor = user_df_floor_mode

                    if (len(site_df_end) > 0)&(len(site_df_start) > 0):
                        if gap_time_start < gap_time_end:
                            corrected_floor = nearest_startpoint.n_floor
                        else:
                            corrected_floor = nearest_endpoint.n_floor
                        
            if pred_floor != corrected_floor:
                tqdm.write(path_id+', '+site_id+': '+str(pred_floor)+' -> '+str(corrected_floor)+': '+str(corrected_floors)+': '+str(gap_time_end)+': '+str(gap_time_start))
                out_df.loc[(out_df.path == path_id),
                                  'floor'] = corrected_floor
                pred_floor = corrected_floor
                counter_floor += 1
        
        if correct_start_end:
            site_df=df_user_train[(df_user_train.site_id==site_id)&(df_user_train.n_floor==pred_floor)]
            user_df=df_user_train[(df_user_train.user_id==user_id)&(df_user_train.n_floor==pred_floor)]
            
            user_df_end = user_df[(user_df.end_time < start_time)]
            if len(user_df_end) > 0:                
                user_df_end = user_df_end[(start_time-user_df_end.end_time<time_threshold_user)]
                if len(user_df_end) > 0:
                    nearest_endpoint = user_df_end.loc[user_df_end.end_time.idxmax()]
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.min()), 'x'] = nearest_endpoint.end_wp_x
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.min()), 'y'] = nearest_endpoint.end_wp_y
                    counter_start += 1
            else:
                site_df_end = site_df[(site_df.end_time < start_time)&(start_time-site_df.end_time<time_threshold_site)]
                if len(site_df_end) > 0:
                    nearest_endpoint = site_df_end.loc[site_df_end.end_time.idxmax()]
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.min()), 'x'] = nearest_endpoint.end_wp_x
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.min()), 'y'] = nearest_endpoint.end_wp_y
                    counter_start += 1
                
            user_df_start = user_df[(end_time < user_df.start_time)]
            if len(user_df_start) > 0:
                user_df_start = user_df_start[(user_df_start.start_time-end_time<time_threshold_user)]
                if len(user_df_start) > 0:
                    nearest_startpoint = user_df_start.loc[user_df_start.start_time.idxmin()]
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.max()), 'x'] = nearest_startpoint.start_wp_x
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.max()), 'y'] = nearest_startpoint.start_wp_y
                    counter_end += 1    
            else:
                site_df_start = site_df[(end_time < site_df.start_time)&(site_df.start_time-end_time<time_threshold_site)]        
                if len(site_df_start) > 0:
                    nearest_startpoint = site_df_start.loc[site_df_start.start_time.idxmin()]
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.max()), 'x'] = nearest_startpoint.start_wp_x
                    out_df.loc[(out_df.path == path_id) & (out_df.timestamp == 
                        out_df[out_df.path == path_id].timestamp.max()), 'y'] = nearest_startpoint.start_wp_y
                    counter_end += 1
            
    out_df = out_df.drop(["site", "path", "timestamp"], axis=1)
    print(str(counter_start) + ' start points are postprocessed.')
    print(str(counter_end) + ' end points are postprocessed.')
    print(str(counter_floor) + ' floors are postprocessed.')
    return out_df

# Postprocessing based on Dijkstra's algorithm
I searched the path with minimal [cost](https://www.kaggle.com/saitodevel01/indoor-post-processing-by-cost-minimization).

In [ ]:
# https://note.nkmk.me/python-scipy-shortest-path/
def get_path(start, goal, pred):
    return get_path_row(start, goal, pred[start])


def get_path_row(start, goal, pred_row):
    path = []
    i = goal
    while i != start and i >= 0:
        path.append(i)
        i = pred_row[i]
    if i < 0:
        return []
    path.append(i)
    return path[::-1]


def shortest_path_postprocessing(input_wps, feature, wps,buffer_x=5, buffer_y=5, abs_ratio=0.05, sensor_mode='both'):
    acce_datas = feature.accelerometer.copy()
    acce_datas = acce_datas.values[:, :4]
    waypoint_timestamps = feature.waypoint.timestamp

    if (sensor_mode == 'rot') or (sensor_mode == 'both'):
        ahrs_datas = feature.rotation_vector.copy()
        ahrs_datas = ahrs_datas.values[:, :4]
        step_timestamps, step_indexs, step_acce_max_mins = compute_steps(
            acce_datas)
        headings = compute_headings(ahrs_datas)
        stride_lengths = compute_stride_length(step_acce_max_mins)
        step_headings = compute_step_heading(step_timestamps, headings)
        rel_pos_rot = compute_rel_positions(stride_lengths, step_headings)
        rel_pos_rot = np.concatenate(
            [np.array([[acce_datas[0, 0], 0, 0]]), rel_pos_rot])
        rel_cumsum_rot = np.cumsum(rel_pos_rot, axis=0)
        rel_t_rot = rel_pos_rot[:, 0]
        rel_x_rot = rel_cumsum_rot[:, 1]
        rel_y_rot = rel_cumsum_rot[:, 2]
        rel_wx_rot = np.interp(waypoint_timestamps, rel_t_rot, rel_x_rot)
        rel_wy_rot = np.interp(waypoint_timestamps, rel_t_rot, rel_y_rot)
        rel_w = np.array([rel_wx_rot, rel_wy_rot]).transpose([1, 0])

    if (sensor_mode == 'mag') or (sensor_mode == 'both'):
        magn_datas = feature.magnetic_field.copy()
        magn_datas = magn_datas.values[:, :4]
        rel_pos_mag = steps_compute_rel_positions(acce_datas, magn_datas)
        rel_pos_mag = np.concatenate(
            [np.array([[acce_datas[0, 0], 0, 0]]), rel_pos_mag])
        rel_cumsum_mag = np.cumsum(rel_pos_mag, axis=0)
        rel_t_mag = rel_pos_mag[:, 0]
        rel_x_mag = rel_cumsum_mag[:, 1]
        rel_y_mag = rel_cumsum_mag[:, 2]
        rel_wx_mag = np.interp(waypoint_timestamps, rel_t_mag, rel_x_mag)
        rel_wy_mag = np.interp(waypoint_timestamps, rel_t_mag, rel_y_mag)
        rel_w = np.array([rel_wx_mag, rel_wy_mag]).transpose([1, 0])

    if sensor_mode == 'both':
        rel1 = rel_pos_rot.copy()
        rel2 = rel_pos_mag.copy()
        rel1[:, 1:] = rel_pos_rot[:, 1:] / 2
        rel2[:, 1:] = rel_pos_mag[:, 1:] / 2
        rel_positions = np.vstack([rel1, rel2])
        rel_pos_rm = rel_positions[np.argsort(rel_positions[:, 0])]
        rel_pos_rm = np.concatenate(
            [np.array([[acce_datas[0, 0], 0, 0]]), rel_pos_rm])
        rel_cumsum_rm = np.cumsum(rel_pos_rm, axis=0)
        rel_t_rm = rel_pos_rm[:, 0]
        rel_x_rm = rel_cumsum_rm[:, 1]
        rel_y_rm = rel_cumsum_rm[:, 2]
        rel_wx_rm = np.interp(waypoint_timestamps, rel_t_rm, rel_x_rm)
        rel_wy_rm = np.interp(waypoint_timestamps, rel_t_rm, rel_y_rm)
        rel_w = np.array([rel_wx_rm, rel_wy_rm]).transpose([1, 0])

    rel_input_wps = np.diff(rel_w, axis=0)

    min_x = input_wps.min(axis=0)[0]-buffer_x
    max_x = input_wps.max(axis=0)[0]+buffer_x
    min_y = input_wps.min(axis=0)[1]-buffer_y
    max_y = input_wps.max(axis=0)[1]+buffer_y
    wps_sub = wps[(wps[:, 0] > min_x) & (max_x > wps[:, 0]) &
                  (wps[:, 1] > min_y) & (max_y > wps[:, 1])]
    if len(wps_sub) < 2:
        min_x = input_wps.min(axis=0)[0]-buffer_x*10
        max_x = input_wps.max(axis=0)[0]+buffer_x*10
        min_y = input_wps.min(axis=0)[1]-buffer_y*10
        max_y = input_wps.max(axis=0)[1]+buffer_y*10
        wps_sub = wps[(wps[:, 0] > min_x) & (max_x > wps[:, 0])
                      & (wps[:, 1] > min_y) & (max_y > wps[:, 1])]
        if len(wps_sub) > 1:
            print('No candidate waypoints for ' +
                  feature.path_id+'. Use buffer x 10. Number of waypoints now: '+str(len(wps_sub)))
        else:
            print('No candidate waypoints for ' +
                  feature.path_id+'. Use all waypoints')
            wps_sub = wps
    len_input_wps = len(input_wps)
    len_wps_sub = len(wps_sub)

    rel_wps = np.tile(wps_sub, (len_wps_sub, 1)) - \
        np.repeat(wps_sub, len_wps_sub, axis=0)
    dist_abs = distance.cdist(input_wps, wps_sub, metric='euclidean')
    dist_rel = distance.cdist(rel_input_wps, rel_wps, metric='euclidean')
    dist_abs = dist_abs**2
    dist_rel = dist_rel**2

    cost_mat_abs = np.zeros(
        [len_input_wps*len_wps_sub+2, len_input_wps*len_wps_sub+2])
    cost_mat_abs[0, 1:len_wps_sub+1] = dist_abs[0, :]
    cost_mat_abs[(len_input_wps-1)*len_wps_sub+1:(len_input_wps)
                 * len_wps_sub+1, len_input_wps*len_wps_sub+1] = 1
    cost_mat_rel = np.zeros(
        [len_input_wps*len_wps_sub+2, len_input_wps*len_wps_sub+2])

    for i in range(len_input_wps-1):
        cost_mat_abs[i*len_wps_sub+1:(i+1)*len_wps_sub+1, (i+1)*len_wps_sub+1:(
            i+2)*len_wps_sub+1] = np.tile(dist_abs[i+1, :], (len_wps_sub, 1)).transpose([1, 0])
        cost_mat_rel[i*len_wps_sub+1:(i+1)*len_wps_sub+1, (i+1)*len_wps_sub+1:(
            i+2)*len_wps_sub+1] = dist_rel[i, :].reshape([len_wps_sub, len_wps_sub])
    cost_mat = cost_mat_abs*abs_ratio+cost_mat_rel*(1-abs_ratio)

    d, p = shortest_path(cost_mat, indices=[
                         0, len_input_wps*len_wps_sub+1], return_predecessors=True)

    path = get_path(0, len_input_wps*len_wps_sub+1, p)
    output_wps = input_wps.copy()
    for i in range(len(output_wps)):
        output_wps[i, :] = wps_sub[(path[i+1]-1) % len_wps_sub]
    return output_wps, rel_w


def plot_shortest_path_postprocessing(input_wps, output_wps, rel_w, ground_truth, feature, wps, aug_wps=np.empty([0,2])):
    plt.plot(rel_w[:, 0]+input_wps[0, 0],
             rel_w[:, 1]+input_wps[0, 1], label="sensor")
    plt.scatter(wps[:, 0], wps[:, 1], label="wps")
    if len(aug_wps)>0:
        plt.scatter(aug_wps[:, 0], aug_wps[:, 1], label="aug_wps")
    plt.plot(input_wps[:, 0], input_wps[:, 1], label="input")
    plt.plot(output_wps[:, 0]-0.25, output_wps[:, 1]-0.25, label="output")
    if len(ground_truth)>0:
        plt.plot(ground_truth[:, 0]+0.25, ground_truth[:, 1]+0.25, label="truth")
        plt.axis([ground_truth.min(axis=0)[0]-20, ground_truth.max(axis=0)[0]+20,
                  ground_truth.min(axis=0)[1]-20, ground_truth.max(axis=0)[1]+20])
        error_input = np.mean(np.sqrt(np.sum((input_wps-ground_truth)**2, axis=1)))
        error_output = np.mean(
            np.sqrt(np.sum((output_wps-ground_truth)**2, axis=1)))
        plt.title(f"{feature.path_id}: {error_input:.5f} -> {error_output:.5f}")
    else:
        temp_arr=np.vstack([input_wps,rel_w+input_wps[0, :]])
        plt.axis([temp_arr.min(axis=0)[0]-20, temp_arr.max(axis=0)[0]+20,
                  temp_arr.min(axis=0)[1]-20, temp_arr.max(axis=0)[1]+20])
        plt.title(f"{feature.path_id}")        
    plt.legend(loc="upper left")
    plt.show()


def evaluate_pp(path_ids, wps_dict, aug_wps_dict, buffer_x, buffer_y, abs_ratio, sensor_mode, noize_std, sample_wps=True, sample_wps_rate=0.9, augment_wps=False):
    error_inputs = []
    error_outputs = []
    error_inputs_p = []
    error_outputs_p = []
    for path_id in tqdm(path_ids):
        feature = pickle_load_dill(out_dir+'features/'+path_id+'.pickle')
        ground_truth = feature.waypoint.values[:, 1:]
        #input_wps = ground_truth + \
        #    (np.random.normal(0, noize_std, (len(ground_truth), 2)))
        input_wps = ground_truth + \
            (np.random.normal(0, noize_std, (len(ground_truth), 2)))+np.ones([len(ground_truth), 2])*np.random.normal(0, noize_std, (1, 2))
        site_id = feature.site_id
        n_floor = feature.n_floor
        wps = wps_dict[site_id][n_floor]
        if sample_wps:
            idx = np.random.choice(np.arange(len(wps)), size=int(len(wps)*sample_wps_rate), replace = False)
            wps=wps[idx,:]            
        if augment_wps:
            aug_wps=aug_wps_dict[site_id][n_floor]
            wps=np.vstack([wps,aug_wps])
        else:
            aug_wps=np.array([0,2])
        output_wps, _ = shortest_path_postprocessing(input_wps, feature, wps, buffer_x, buffer_y, abs_ratio, sensor_mode)
        error_input = np.sqrt(np.sum((input_wps-ground_truth)**2, axis=1))
        error_output = np.sqrt(np.sum((output_wps-ground_truth)**2, axis=1))
        error_input_p = np.mean(
            np.sqrt(np.sum((input_wps-ground_truth)**2, axis=1)))
        error_output_p = np.mean(
            np.sqrt(np.sum((output_wps-ground_truth)**2, axis=1)))
        error_inputs.append(error_input)
        error_outputs.append(error_output)
        error_inputs_p.append([error_input_p])
        error_outputs_p.append([error_input_p])

    error_inputs = np.concatenate(error_inputs)
    error_outputs = np.concatenate(error_outputs)
    error_inputs_p = np.concatenate([error_inputs_p])
    error_outputs_p = np.concatenate([error_outputs_p])
    return error_inputs.mean(), error_outputs.mean(), error_inputs_p, error_outputs_p


In [ ]:
count_num_wps_df=pd.DataFrame(test_meta.value_counts('num_wps')).reset_index()
count_num_wps_df.columns=['num_wps','num_wps_counts']
train_meta_sub=fast_left_merge(train_meta_sub, count_num_wps_df, key='num_wps')
path_ids = train_meta_sub.sample(100, replace=False, weights=train_meta_sub.num_wps_counts).path_id.values

These are the examples of the results of shortest path postprocessing

In [ ]:
sample_wps=True
sample_wps_rate=0.9
augment_wps=True
noize_std=7
buffer_x=5
buffer_y=5
abs_ratio=0.05 
sensor_mode='both'
#for noize_std in noize_stds:
for path_id in path_ids[:10]:
#path_ids = train_meta_sub.sample(1000, replace=False, weights=train_meta_sub.num_wps_counts).path_id.values
#for path_id in path_ids:
    feature = pickle_load_dill(out_dir+'features/'+path_id+'.pickle')
    ground_truth = feature.waypoint.values[:, 1:]
    site_id = feature.site_id
    n_floor = feature.n_floor
    wps = wps_dict[site_id][n_floor]
    if sample_wps:
        idx = np.random.choice(np.arange(len(wps)), size=int(len(wps)*sample_wps_rate), replace = False)
        wps=wps[idx,:]
    input_wps = ground_truth + \
        (np.random.normal(0, noize_std, (len(ground_truth), 2)))
    
    if augment_wps:
        aug_wps=aug_wps_dict[feature.site_id][feature.n_floor]
        wps=np.vstack([wps,aug_wps])
    else:
        aug_wps=np.array([0,2])
    output_wps, rel_w = shortest_path_postprocessing(input_wps, feature, wps, buffer_x, buffer_y, abs_ratio, sensor_mode)

    plot_shortest_path_postprocessing(
        input_wps, output_wps, rel_w, ground_truth, feature, wps, aug_wps)

    #print(path_id)
    #print(input_wps)
    #print(output_wps)
    #print(ground_truth)
    #error_input = np.mean(np.sqrt(np.sum((input_wps-ground_truth)**2, axis=1)))
    #error_output = np.mean(
    #    np.sqrt(np.sum((output_wps-ground_truth)**2, axis=1)))
    #print(f"{path_id}: {error_input:.5f} -> {error_output:.5f}")

In [ ]:
def correct_pred_shortest_path(sub_df, wps_dict, aug_wps_dict=aug_wps_dict, buffer_x=5, buffer_y=5, abs_ratio=0.05, sensor_mode='both', plot_results=True, augment_wps=False):
    tmp = sub_df['site_path_timestamp'].apply(
        lambda s: pd.Series(s.split('_')))
    sub_df['site'] = tmp[0]
    sub_df['path'] = tmp[1]
    sub_df['timestamp'] = tmp[2].astype(float)

    dfs = []
    for path_id, df in tqdm(sub_df.groupby('path')):
        input_wps = df[['x', 'y']].values.copy()
        feature = pickle_load_dill(out_dir+'features/'+path_id+'.pickle')
        feature.path_id=path_id
        site_id = feature.site_id
        n_floor = int(df.floor.mean())
        wps = wps_dict[site_id][n_floor]
        if augment_wps:
            aug_wps=aug_wps_dict[site_id][n_floor]
            wps=np.vstack([wps,aug_wps])
        else:
            aug_wps=np.empty([0,2])
        output_wps, rel_w = shortest_path_postprocessing(input_wps, feature, wps, buffer_x, buffer_y, abs_ratio, sensor_mode)
        if plot_results:
            plot_shortest_path_postprocessing(input_wps, output_wps, rel_w, np.empty([0,2]), feature, wps, aug_wps)
        df[['x', 'y']] = output_wps
        dfs.append(df)
    processed_df = pd.concat(dfs).sort_values('site_path_timestamp')
    processed_df = processed_df.drop(["site", "path", "timestamp"], axis=1)
    return processed_df


In [ ]:
nigel_pred_df=pd.read_csv('../input/simple-99-accurate-floor-model/submission.csv')

In [ ]:
submission_df = pd.read_csv('../input/ensemble-best-lbs/submission.csv')
submission_df_l_f=correct_pred_leak(submission_df,df_user,nigel_pred_df,correct_start_end=False,correct_floor=True, if_all_count_one='nigel',time_threshold_site=10000,time_threshold_user=10000)
submission_df_l_f_pp = correct_pred_shortest_path(submission_df_l_f, wps_dict, plot_results=False,augment_wps=False)
submission_df_l_f_pp.to_csv('submission_df_l_f_pp.csv', index=False)
submission_df_l_f_pp_aug = correct_pred_shortest_path(submission_df_l_f, wps_dict, plot_results=False,augment_wps=True)
submission_df_l_f_pp_aug.to_csv('submission_df_l_f_pp_aug.csv', index=False)

# Visualization


In [ ]:
train_waypoints = pd.read_csv('/kaggle/input/indoor-location-train-waypoints/train_waypoints.csv')
sub_df=submission_df
tmp = sub_df['site_path_timestamp'].apply(
    lambda s: pd.Series(s.split('_')))
sub_df['site'] = tmp[0]
sub_df['path'] = tmp[1]
sub_df['timestamp'] = tmp[2].astype(float)
all_sites = generate_target_sites(submission_df)
site_floors_dict = generate_site_floors_dict(submission_df)
site_floors_dict

submission_df = sub_process(submission_df, train_waypoints)
submission_df_l_f_pp = sub_process(submission_df_l_f_pp, train_waypoints)
submission_df_l_f_pp_aug = sub_process(submission_df_l_f_pp_aug, train_waypoints)

sites1 = all_sites[:8]
sites2 = all_sites[8:16]
sites3 = all_sites[16:]

In [ ]:
plot_sites(sites1, submission_df, submission_df_l_f_pp, submission_df_l_f_pp_aug)

In [ ]:
plot_sites(sites2, submission_df, submission_df_l_f_pp, submission_df_l_f_pp_aug)

In [ ]:
plot_sites(sites3, submission_df, submission_df_l_f_pp, submission_df_l_f_pp_aug)